# Airbyte Salesforce | Airbyte Salesforce

> [Airbyte](https://github.com/airbytehq/airbyte)は、API、データベース、ファイルからデータウェアハウスやデータレイクへのELTパイプラインのためのデータ統合プラットフォームです。データウェアハウスとデータベースへのELTコネクターのカタログが最も豊富です。>> > [Airbyte](https://github.com/airbytehq/airbyte) is a data integration platform for ELT pipelines from APIs, databases & files to warehouses & lakes. It has the largest catalog of ELT connectors to data warehouses and databases.このローダーはSalesforceコネクタをドキュメントローダーとして公開し、様々なSalesforceオブジェクトをドキュメントとして読み込むことを可能にします。> This loader exposes the Salesforce connector as a document loader, allowing you to load various Salesforce objects as documents.

## Installation | インストール

まず、`airbyte-source-salesforce` Pythonパッケージをインストールする必要があります。> First, you need to install the `airbyte-source-salesforce` python package.

In [ ]:
#!pip install airbyte-source-salesforce

## Example | 例

リーダーの設定方法の詳細については、[Airbyteのドキュメントページ](https://docs.airbyte.com/integrations/sources/salesforce/)をご覧ください。設定オブジェクトが準拠すべきJSONスキーマはGithubにあります: <https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-salesforce/source_salesforce/spec.yaml>。> Check out the [Airbyte documentation page](https://docs.airbyte.com/integrations/sources/salesforce/) for details about how to configure the reader.> The JSON schema the config object should adhere to can be found on Github: <https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-salesforce/source_salesforce/spec.yaml>.一般的な形は次のようになります：> The general shape looks like this:```python{  "client_id": "<oauth client id>",  "client_secret": "<oauth client secret>",  "refresh_token": "<oauth refresh token>",  "start_date": "<date from which to start retrieving records from in ISO format, e.g. 2020-10-20T00:00:00Z>",  "is_sandbox": False, # set to True if you're using a sandbox environment  "streams_criteria": [ # Array of filters for salesforce objects that should be loadable    {"criteria": "exacts", "value": "Account"}, # Exact name of salesforce object    {"criteria": "starts with", "value": "Asset"}, # Prefix of the name    # Other allowed criteria: ends with, contains, starts not with, ends not with, not contains, not exacts  ],}```デフォルトでは、すべてのフィールドはドキュメントのメタデータとして保存され、テキストは空の文字列に設定されています。リーダーから返されるドキュメントを変換することで、ドキュメントのテキストを構築してください。> By default all fields are stored as metadata in the documents and the text is set to an empty string. Construct the text of the document by transforming the documents returned by the reader.

In [ ]:
from langchain.document_loaders.airbyte import AirbyteSalesforceLoader

config = {
    # your salesforce configuration
}

loader = AirbyteSalesforceLoader(
    config=config, stream_name="asset"
)  # check the documentation linked above for a list of all streams

これで、通常の方法でドキュメントを読み込むことができます> Now you can load documents the usual way

In [ ]:
docs = loader.load()

`load`はリストを返すため、すべてのドキュメントが読み込まれるまで処理がブロックされます。このプロセスをより良くコントロールするために、イテレータを返す`lazy_load`メソッドを使用することもできます：> As `load` returns a list, it will block until all documents are loaded. To have better control over this process, you can also you the `lazy_load` method which returns an iterator instead:

In [ ]:
docs_iterator = loader.lazy_load()

デフォルトではページの内容は空であり、メタデータオブジェクトにはレコードからのすべての情報が含まれていることに注意してください。異なる形式でドキュメントを作成するには、ローダーを作成する際に record\_handler 関数を渡す必要があります：> Keep in mind that by default the page content is empty and the metadata object contains all the information from the record. To create documents in a different, pass in a record\_handler function when creating the loader:

In [ ]:
from langchain.docstore.document import Document


def handle_record(record, id):
    return Document(page_content=record.data["title"], metadata=record.data)


loader = AirbyteSalesforceLoader(
    config=config, record_handler=handle_record, stream_name="asset"
)
docs = loader.load()

## Incremental loads | インクリメンタルロード一部のストリームではインクリメンタルローディングが可能です。これは、ソースが同期されたレコードを追跡し、それらを再度ロードしないことを意味します。これは、データ量が多く、頻繁に更新されるソースにとって有用です。> Some streams allow incremental loading, this means the source keeps track of synced records and won't load them again. This is useful for sources that have a high volume of data and are updated frequently.これを利用するためには、ローダーの `last_state` プロパティを保存し、ローダーを再作成する際にそれを渡すことが必要です。これにより、新しいレコードだけがロードされることが保証されます。> To take advantage of this, store the `last_state` property of the loader and pass it in when creating the loader again. This will ensure that only new records are loaded.

In [ ]:
last_state = loader.last_state  # store safely

incremental_loader = AirbyteSalesforceLoader(
    config=config, stream_name="asset", state=last_state
)

new_docs = incremental_loader.load()